In [22]:
import time
import numpy as np

In [29]:
# open the sample file used
file = open('Sample/sample-input.txt')
  
# read the content of the file opened
content = file.readlines()

filtered_content = []
boards = []
# fuel = {}
for line in content:
    if line[0] != '#' and line[0] != '\n':
        filtered_content.append(line.strip())
        
for line in filtered_content: 
    line_split = line.split() 
    p = line_split[0]
    
    # get puzzle in array form 
    board = []
    for i in range(0, 35, 6):
        board.append(p[i:i+6])
    boards.append(board)


boards

[['BBIJ..', '..IJCC', '..IAAM', 'GDDK.M', 'GH.KL.', 'GHFFL.'],
 ['..I...', 'BBI.K.', 'GHAAKL', 'GHDDKL', 'G..JEE', 'FF.J..'],
 ['JBBCCC', 'JDD..M', 'JAAL.M', 'FFKL.N', '..KGGN', '.HH...'],
 ['BBB..M', 'CCDD.M', 'AAKL.M', 'J.KLEE', 'J.GG..', 'JHHHII'],
 ['IJBBCC', 'IJDDL.', 'IJAAL.', 'EEK.L.', '..KFF.', '.GGHH.'],
 ['BB.G.H', 'E..G.H', 'EAAG.I', '..FCCI', 'DDF..I', '..F...']]

In [5]:
filtered_content

['BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL.',
 '..I...BBI.K.GHAAKLGHDDKLG..JEEFF.J..',
 'JBBCCCJDD..MJAAL.MFFKL.N..KGGN.HH...',
 'BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII J0 B4',
 'IJBBCCIJDDL.IJAAL.EEK.L...KFF..GGHH. F0 G6',
 'BB.G.HE..G.HEAAG.I..FCCIDDF..I..F...']

In [6]:
def get_board(puzzle): 
    puzzle_split = puzzle.split() 
    p = puzzle_split[0]
    
    board = []
    for i in range(0, 35, 6):
        board.append(p[i:i+6])
    
    return board

In [7]:
get_board(filtered_content[3])

['BBB..M', 'CCDD.M', 'AAKL.M', 'J.KLEE', 'J.GG..', 'JHHHII']

In [8]:
def get_fuel(puzzle): 
    fuel = {}
    puzzle_split = puzzle.split() 
    p = puzzle_split[0]
    
    for f in puzzle_split[1:]: 
        fuel[f[0]] = f[1]
    
    for c in p: 
        if c.isalpha() and c not in fuel: 
            fuel[c] = 100
            
    return fuel

In [9]:
get_fuel(filtered_content[3])

{'J': '0',
 'B': '4',
 'M': 100,
 'C': 100,
 'D': 100,
 'A': 100,
 'K': 100,
 'L': 100,
 'E': 100,
 'G': 100,
 'H': 100,
 'I': 100}

In [10]:
def print_board(board):
    print('   1 2 3 4 5 6')
    print('   ------------')
    i = 0
    for row in board:
        i += 1
        print(str(i) + '|', end=' ')
        for c in row:
            print(c, end=' ')
        print()

In [11]:
print_board(boards[0])

   1 2 3 4 5 6
   ------------
1| B B I J . . 
2| . . I J C C 
3| . . I A A M 
4| G D D K . M 
5| G H . K L . 
6| G H F F L . 


In [12]:
# THIS IS THE PUZZLE THAT WILL BE USED 
puzzle = filtered_content[0]
board = get_board(filtered_content[0])
fuel = get_fuel(filtered_content[0])
print_board(board)
fuel

   1 2 3 4 5 6
   ------------
1| B B I J . . 
2| . . I J C C 
3| . . I A A M 
4| G D D K . M 
5| G H . K L . 
6| G H F F L . 


{'B': 100,
 'I': 100,
 'J': 100,
 'C': 100,
 'A': 100,
 'M': 100,
 'G': 100,
 'D': 100,
 'K': 100,
 'H': 100,
 'L': 100,
 'F': 100}

In [13]:
print(board[0][0])
print(board[0][1])

print(board[0][2])
print(board[1][2])
print(board[2][2])

B
B
I
I
I


# 1. UCS

In [14]:
class Car: 
    def __init__(self, position, length, orientation): 
        self.position = position 
        self.length = length 
        self.orientation = orientation 

In [16]:
# go through the board, identify moves, make them into states 
# for ONE BOARD 
def get_cars(board):
    cars = {}
    for i in range(6): # iterate over each row
        for j in range(6): # iterate over each column
            if board[i][j].isalpha(): # B # I
                if j < 5 and board[i][j+1] == board[i][j]: # B == B # I != J skip
                    posi = j+1 # posi = 1
                    while posi < 5 and board[i][posi] == board[i][j]: # board[0][1] = B, board[0][2] = I, not equal, break
                        posi += 1 # posi = 2
                    
                    if board[i][j] not in cars:
                        cars[board[i][j]] = Car([i, j], posi - j, 'h') # map car B to its parameters 

                    j = posi - 1 # go to last position of car just visited

                elif i < 5 and board[i+1][j] == board[i][j]: # I = I
                    posi = i+1 # posi = 1
                    while posi < 5 and board[posi][j] == board[i][j]: # I = I, I = I, I != D, exit loop
                        posi += 1 # posi = 2, posi = 3
                    if board[i][j] not in cars:
                        cars[board[i][j]] = Car([i, j], posi - i, 'v')
    return cars

print_board(board)
print()
cars = get_cars(board)
for car in cars: 
    print(car + ' [' + str(cars[car].position[0]) + ', ' + str(cars[car].position[1]) + '] ' + str(cars[car].length) + ' ' + cars[car].orientation)

   1 2 3 4 5 6
   ------------
1| B B I J . . 
2| . . I J C C 
3| . . I A A M 
4| G D D K . M 
5| G H . K L . 
6| G H F F L . 

B [0, 0] 2 h
I [0, 2] 3 v
J [0, 3] 2 v
C [1, 4] 1 h
A [2, 3] 2 h
M [2, 5] 2 v
G [3, 0] 2 v
D [3, 1] 2 h
K [3, 3] 2 v
H [4, 1] 1 v
L [4, 4] 1 v
F [5, 2] 2 h


In [ ]:
# go through the board, identify moves, make them into states 
# for ONE BOARD 
def get_cars(board):
    cars = {}
    for i in range(6): # iterate over each row
        for j in range(6): # iterate over each column
            if board[i][j].isalpha(): # B # I
                if j < 5 and board[i][j+1] == board[i][j]: # B == B # I != J skip
                    posi = j+1 # posi = 1
                    while posi < 5 and board[i][posi] == board[i][j]: # board[0][1] = B, board[0][2] = I, not equal, break
                        posi += 1 # posi = 2
                    
                    if board[i][j] not in cars:
                        cars[board[i][j]] = Car([i, j], posi - j, 'h') # map car B to its parameters 

                    j = posi - 1 # go to last position of car just visited

                elif i < 5 and board[i+1][j] == board[i][j]: # I = I
                    posi = i+1 # posi = 1
                    while posi < 5 and board[posi][j] == board[i][j]: # I = I, I = I, I != D, exit loop
                        posi += 1 # posi = 2, posi = 3
                    if board[i][j] not in cars:
                        cars[board[i][j]] = Car([i, j], posi - i, 'v')
    return cars

print_board(board)
print()
cars = get_cars(board)
for car in cars: 
    print(car + ' [' + str(cars[car].position[0]) + ', ' + str(cars[car].position[1]) + '] ' + str(cars[car].length) + ' ' + cars[car].orientation)

In [ ]:
# identify potential moves
def moves_identified(board):
    cars = get_cars(board)
    for car, params in cars.items(): 
        i = params.position[0]
        j = params.position[j]
        if params.orientation == 'h': 
            if j-1 >= 0 and board[i][j-1] == '.': 
                # handle this case 
            if j+1 < 6 and board[i][j+1] == ',': 
                #handle this case 
        elif params.orientation == 'v': 
            if i-1 >= 0 and board[i-1][j] == '.': 
                # handle this case 
            if i+1 < 6 and board[i-1][j] == '.': 
                posi = i-1 
                while posi < 6 and board[posi][i] == '.': 
                    # ** LEFT OFF HERE ** 
                    # figure out how to keep track of potential moves. make a state out of it directly 
                    # or keep enough info stored to make all the states at once later? 

In [ ]:
class BoardState:
    def __init__(self): 
        

In [ ]:
class Game:
    def __init__(self):
        self.initialize_game()
        
    def initialize_game(self):
        self.current_state = [['.','.','.'],
                              ['.','.','.'],
                              ['.','.','.']]
        # Player X always plays first
        self.player_turn = 'X'

    # draw the board
    def draw_board(self):
        print()
        for y in range(0, 3):
            for x in range(0, 3):
                print(F'{self.current_state[x][y]}', end="")
            print()
        print()
        
    # validates if coordinates are valid
    def is_valid(self, px, py):
        if px < 0 or px > 2 or py < 0 or py > 2:
            return False
        elif self.current_state[px][py] != '.':
            return False
        else:
            return True

    # check if the board contains a win
    def is_end(self):
        # Vertical win
        for i in range(0, 3):
            if (self.current_state[0][i] != '.' and
                self.current_state[0][i] == self.current_state[1][i] and
                self.current_state[1][i] == self.current_state[2][i]):
                return self.current_state[0][i]
        # Horizontal win
        for i in range(0, 3):
            if (self.current_state[i] == ['X', 'X', 'X']):
                return 'X'
            elif (self.current_state[i] == ['O', 'O', 'O']):
                return 'O'
        # Main diagonal win
        if (self.current_state[0][0] != '.' and
            self.current_state[0][0] == self.current_state[1][1] and
            self.current_state[0][0] == self.current_state[2][2]):
            return self.current_state[0][0]
        # Second diagonal win
        if (self.current_state[0][2] != '.' and
            self.current_state[0][2] == self.current_state[1][1] and
            self.current_state[0][2] == self.current_state[2][0]):
            return self.current_state[0][2]
        # Is whole board full?
        for i in range(0, 3):
            for j in range(0, 3):
                # There's an empty field, we continue the game
                if (self.current_state[i][j] == '.'):
                    return None
        # It's a tie!
        return '.'

    # check of end of game
    def check_end(self):
        self.result = self.is_end()
        # Printing the appropriate message if the game has ended
        if self.result != None:
            if self.result == 'X':
                print('The winner is X!')
            elif self.result == 'O':
                print('The winner is O!')
            elif self.result == '.':
                print("It's a tie!")
            self.initialize_game()
        return self.result

    # get user inputs (X and Y corrdinates)
    def input_move(self):
        while True:
            print(F'Player {self.player_turn}, enter your move:')
            px = int(input('enter the x coordinate: '))
            py = int(input('enter the y coordinate: '))
            if self.is_valid(px, py):
                return (px,py)
            else:
                print('The move is not valid! Try again.')

    # alternate players after a turn
    def switch_player(self):
        if self.player_turn == 'X':
            self.player_turn = 'O'
        elif self.player_turn == 'O':
            self.player_turn = 'X'
        return self.player_turn

    # THE BUG HAS BEEN FIXED
    def minimax(self, maximize=False):
        # Minimizing for 'X' and maximizing for 'O'
        # Possible values are:
        # -1 - win for 'X'
        # 0  - a tie
        # 1  - loss for 'X'
        # We're initially setting it to 2 or -2 as worse than the worst case:
        value = 2
        if max:
            value = -2
        x = None
        y = None
        result = self.is_end()
        if result == 'X':
            return (-1, x, y)
        elif result == 'O':
            return (1, x, y)
        elif result == '.':
            return (0, x, y)
        # evaluate all possible children states and pick the optimal one
        # depending on whether we are maximizing or minimizing
        for i in range(0, 3):
            for j in range(0, 3):
                if self.current_state[i][j] == '.':
                    if max:
                        # create a child state
                        self.current_state[i][j] = 'O'
                        # evaluate it
                        (v, _, _) = self.minimax(maximize=False)
                        # keep its coordinates if best so far
                        if v > value:
                            value = v
                            x = i
                            y = j
                    else:
                        # create a child state
                        self.current_state[i][j] = 'X'
                        # evaluate it
                        (v, _, _) = self.minimax(maximize=True)
                        # keep its coordinates if best so far
                        if v < value:
                            value = v
                            x = i
                            y = j
                    # undo the child state <---------
                    self.current_state[i][j] = '.'
        # return best coordinates and associated value
        return (value, x, y)

    def play(self,is_human_x=True,is_human_o=False):
        while True:
            self.draw_board()
            if self.check_end():
                return
            # determine next move (human or AI)
            if (self.player_turn == 'X' and is_human_x) or (self.player_turn == 'O' and is_human_o):
                (x,y) = self.input_move()
            else:
                start = time.time()
                # we maximize for player O and minimize for player X
                (_, x, y) = self.minimax(maximize=self.player_turn == 'O')
                end = time.time()
                print(F'Evaluation time: {round(end - start, 7)}s')
                print(F'Player {self.player_turn} under AI control plays: x = {x}, y = {y}')
            # play the move
            self.current_state[x][y] = self.player_turn
            self.switch_player()

g = Game()